In [ ]:
# ✅ Step 1: Imports and Setup
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2, VGG16, EfficientNetB0
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

In [ ]:
# ✅ Step 2: Load and Preprocess Dataset
def load_data_from_nested_dirs(data_dir):
    images = []
    labels = []
    for split in ['Training', 'Testing']:
        split_dir = os.path.join(data_dir, split)
        if not os.path.exists(split_dir):
            continue
        for class_name in os.listdir(split_dir):
            class_path = os.path.join(split_dir, class_name)
            if not os.path.isdir(class_path):
                continue
            for img_file in os.listdir(class_path):
                if img_file.endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(class_path, img_file)
                    label = class_name.lower().replace('no_tumor', 'notumor').replace('_tumor', '')
                    images.append(img_path)
                    labels.append(label)
    return images, labels

base_dir = "/kaggle/input"
dataset1 = os.path.join(base_dir, "brain-tumor-mri-dataset")
dataset2 = os.path.join(base_dir, "brain-tumor-classification-mri")

images1, labels1 = load_data_from_nested_dirs(dataset1)
images2, labels2 = load_data_from_nested_dirs(dataset2)

images = images1 + images2
labels = labels1 + labels2

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

X_train, X_val, y_train, y_val = train_test_split(images, labels_encoded, test_size=0.2, stratify=labels_encoded, random_state=42)


In [ ]:
# ✅ Step 3: Resize and Normalize
IMG_SIZE = (224, 224)

def load_and_resize(img_paths):
    return np.array([cv2.resize(cv2.imread(path), IMG_SIZE) for path in img_paths])

X_train_resized = load_and_resize(X_train)
X_val_resized = load_and_resize(X_val)

y_train_cat = to_categorical(y_train, num_classes=4)
y_val_cat = to_categorical(y_val, num_classes=4)

In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image

# Define dataset paths
dataset1_path = "/kaggle/input/brain-tumor-mri-dataset"
dataset2_path = "/kaggle/input/brain-tumor-classification-mri"

# Class name mapping for standardization
name_map = {
    'glioma_tumor': 'glioma',
    'meningioma_tumor': 'meningioma',
    'pituitary_tumor': 'pituitary',
    'no_tumor': 'notumor',
    'glioma': 'glioma',
    'meningioma': 'meningioma',
    'pituitary': 'pituitary',
    'notumor': 'notumor'
}

# Function to collect metadata from a dataset
def collect_dataset_info(base_path, dataset_name):
    data = []

    for split in ['Training', 'Testing']:
        split_path = os.path.join(base_path, split)
        if not os.path.exists(split_path):
            continue

        for class_folder in os.listdir(split_path):
            class_path = os.path.join(split_path, class_folder)
            if not os.path.isdir(class_path):
                continue

            class_name = name_map.get(class_folder, class_folder)

            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_file)
                    try:
                        with Image.open(img_path) as img:
                            width, height = img.size
                            mode = img.mode
                        data.append({
                            'dataset': dataset_name,
                            'split': split,
                            'class': class_name,
                            'image_name': img_file,
                            'path': img_path,
                            'width': width,
                            'height': height,
                            'mode': mode
                        })
                    except Exception as e:
                        print(f"Error reading {img_path}: {e}")
    return pd.DataFrame(data)

# Load metadata from both datasets
df1 = collect_dataset_info(dataset1_path, "dataset1")
df2 = collect_dataset_info(dataset2_path, "dataset2")

# Merge them
df = pd.concat([df1, df2], ignore_index=True)

# ---- EDA Starts ----

print("Total images in merged dataset:", len(df))
print(df.head())

# 📊 Class distribution by split
plt.figure(figsize=(10, 5))
sns.countplot(data=df, x='class', hue='split', palette='Set2')
plt.title("Image Distribution by Class and Split")
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()

# 📏 Image size distributions
plt.figure(figsize=(8, 5))
sns.histplot(df['width'], bins=30, color='skyblue', label='Width', kde=True)
sns.histplot(df['height'], bins=30, color='orange', label='Height', kde=True)
plt.legend()
plt.title("Image Dimension Distribution")
plt.tight_layout()
plt.show()

# 🖼️ Show example images
def show_samples(df, split='Training'):
    classes = df['class'].unique()
    plt.figure(figsize=(15, 8))
    for i, cls in enumerate(classes):
        sample = df[(df['class'] == cls) & (df['split'] == split)].iloc[0]
        img = Image.open(sample['path'])
        plt.subplot(2, len(classes)//2 + 1, i+1)
        plt.imshow(img)
        plt.title(f"{split}: {cls}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Show sample images
show_samples(df, split='Training')
show_samples(df, split='Testing')


In [ ]:
# First, make a copy of the merged dataset
balanced_2500_df = pd.DataFrame()

target_count = 2500

for cls in df['class'].unique():
    class_df = df[df['class'] == cls]
    
    # Downsample to 2500 if necessary
    sampled_df = class_df.sample(n=target_count, random_state=42)
    
    balanced_2500_df = pd.concat([balanced_2500_df, sampled_df], ignore_index=True)

# Shuffle the final DataFrame
balanced_2500_df = balanced_2500_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Confirm the result
print("✅ Class-wise Count After Forcing All to 2500:")
print(balanced_2500_df['class'].value_counts())


plt.figure(figsize=(8, 5))
sns.countplot(data=balanced_2500_df, x='class', palette='pastel')
plt.title("Balanced Dataset with 2500 Images per Class")
plt.xlabel("Tumor Class")
plt.ylabel("Image Count")
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd

# Split 2500 images per class into train/test (80/20)
split_balanced = []

# Define the desired class order
class_order = ['glioma', 'meningioma', 'pituitary', 'notumor']

# Ensure consistent lowercase class names
balanced_2500_df['class'] = balanced_2500_df['class'].str.lower()

for cls in class_order:
    class_df = balanced_2500_df[balanced_2500_df['class'] == cls]
    
    train, test = train_test_split(class_df, test_size=0.2, random_state=42, shuffle=True)
    
    train = train.copy()
    test = test.copy()
    train['split'] = 'Training'
    test['split'] = 'Testing'
    
    split_balanced.extend([train, test])

# Merge all splits
split_df = pd.concat(split_balanced, ignore_index=True)

# Convert 'class' column to ordered categorical to control plot order
split_df['class'] = pd.Categorical(split_df['class'], categories=class_order, ordered=True)

# Plot
plt.figure(figsize=(10, 6))
sns.countplot(data=split_df, x='class', hue='split', palette='Set2')

plt.title("Balanced Dataset - Train/Test Split")
plt.xlabel("Tumor Class")
plt.ylabel("Number of Images")
plt.tight_layout()
plt.show()


In [ ]:
# ✅ Step 4: Compute Class Weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
def build_custom_cnn():
    inputs = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D()(x)

    x = Dropout(0.4)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(4, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model



In [ ]:
# ✅ Step 6: Build Transfer Learning Wrapper
def build_transfer_model(base, preprocess):
    base_model = base(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(4, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [ ]:
# ✅ Step 7: Train All Models
models = {
    "CustomCNN": build_custom_cnn(),
    "VGG16": build_transfer_model(VGG16, vgg16_preprocess),
    "MobileNetV2": build_transfer_model(MobileNetV2, mobilenet_preprocess),
    "EfficientNetB0": build_transfer_model(EfficientNetB0, efficientnet_preprocess)
}

histories = {}
model_preds = {}
model_scores = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

    if name == "CustomCNN":
        X_train_input = X_train_resized / 255.
        X_val_input = X_val_resized / 255.
    elif name == "VGG16":
        X_train_input = vgg16_preprocess(X_train_resized.copy())
        X_val_input = vgg16_preprocess(X_val_resized.copy())
    elif name == "MobileNetV2":
        X_train_input = mobilenet_preprocess(X_train_resized.copy())
        X_val_input = mobilenet_preprocess(X_val_resized.copy())
    elif name == "EfficientNetB0":
        X_train_input = efficientnet_preprocess(X_train_resized.copy())
        X_val_input = efficientnet_preprocess(X_val_resized.copy())

    history = model.fit(
        X_train_input, y_train_cat,
        validation_data=(X_val_input, y_val_cat),
        epochs=50,
        batch_size=32,
        class_weight=class_weights_dict,
        callbacks=[EarlyStopping(patience=9, restore_best_weights=True), ReduceLROnPlateau(patience=5)],
        verbose=2
    )
    histories[name] = history

    y_pred = model.predict(X_val_input)
    y_pred_classes = np.argmax(y_pred, axis=1)
    acc = np.mean(y_pred_classes == y_val)
    model_preds[name] = y_pred
    model_scores[name] = acc

    print(f"\n{name} Accuracy: {acc * 100:.2f}%")
    print(classification_report(y_val, y_pred_classes, target_names=le.classes_))
    cm = confusion_matrix(y_val, y_pred_classes)
    ConfusionMatrixDisplay(cm, display_labels=le.classes_).plot(cmap='Blues')
    plt.title(f"{name} Confusion Matrix")
    plt.show()


In [ ]:
# ✅ Step 8: Ensemble Prediction (Soft Voting)
all_preds = np.mean(list(model_preds.values()), axis=0)
y_ensemble = np.argmax(all_preds, axis=1)
ensemble_acc = np.mean(y_ensemble == y_val)
print(f"\n📌 Ensemble Accuracy: {ensemble_acc * 100:.2f}%")
print(classification_report(y_val, y_ensemble, target_names=le.classes_))
cm_ensemble = confusion_matrix(y_val, y_ensemble)
ConfusionMatrixDisplay(cm_ensemble, display_labels=le.classes_).plot(cmap='Blues')
plt.title("🧠 Ensemble Confusion Matrix")
plt.show()

In [ ]:
# ✅ Step 9: Grad-CAM Visualization for All Models (Kaggle-compatible)
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Patch CustomCNN so it's 'called'
_ = models["CustomCNN"].predict(np.expand_dims(X_val_resized[0] / 255., axis=0))


# ✅ Grad-CAM Generator Function (Robust for all models)
def generate_grad_cam(model, image, label_index, preprocess=None, model_name="Model"):
    # Preprocess image
    img_tensor = np.expand_dims(image, axis=0)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    # 🛠 Build/Call Sequential models manually to fix the "never been called" error
    try:
        _ = model.predict(img_tensor)
    except:
        model.build((None, 224, 224, 3))
        _ = model.predict(img_tensor)

    # 🔍 Find last Conv2D layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break
    if not last_conv_layer:
        raise ValueError(f"No Conv2D layer found in model {model_name}")

    # 🎯 Build Grad-CAM model
    grad_model = Model(
        inputs=model.input,
        outputs=[model.get_layer(last_conv_layer).output, model.output]
    )

    # 🔥 Compute gradients
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)[0]
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    conv_outputs = conv_outputs[0]

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + tf.keras.backend.epsilon())
    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)

    superimposed_img = cv2.addWeighted(image.astype(np.uint8), 0.6, heatmap_colored, 0.4, 0)
    return superimposed_img

# ✅ Preprocessing for each model
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess,
    
}

# ✅ Sample image from validation set
sample_img = X_val_resized[0]
true_label = y_val[0]

# 🔥 Generate Grad-CAM visualizations
plt.figure(figsize=(20, 5))
for i, (name, model) in enumerate(models.items()):
    print(f"🔍 Generating Grad-CAM for {name}...")
    try:
        cam_img = generate_grad_cam(model, sample_img, true_label, preprocess=preprocessors[name], model_name=name)
        plt.subplot(1, 4, i + 1)
        plt.imshow(cam_img)
        plt.title(f"{name} Grad-CAM")
        plt.axis('off')
    except Exception as e:
        print(f"❌ Failed for {name}: {e}")

plt.tight_layout()
plt.show()


In [ ]:
# ✅ Step 10: Save Best Model
best_model_name = max(model_scores, key=model_scores.get)
print(f"\n✅ Best Performing Model: {best_model_name} with Accuracy: {model_scores[best_model_name] * 100:.2f}%")
models[best_model_name].save(f"best_model_{best_model_name}.h5")


In [ ]:
# ✅ Step 11: Display All Accuracies
print("\n📊 All Model Accuracies:")
for model_name, score in model_scores.items():
    print(f"{model_name}: {score * 100:.2f}%")
print(f"Ensemble: {ensemble_acc * 100:.2f}%")


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

# Grad-CAM Function
def generate_grad_cam(model, image, label_index, preprocess=None):
    img_tensor = np.expand_dims(image, axis=0)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    # Detect last Conv layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break

    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image.astype(np.uint8), 0.6, heatmap_colored, 0.4, 0)
    return superimposed_img

# 🔍 Sample image (1st from validation set)
sample_img = X_val_resized[0]
true_label = y_val[0]

# Define preprocessors
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess
}

# 🔥 Plot Grad-CAMs
plt.figure(figsize=(16, 8))
for i, (name, model) in enumerate(models.items()):
    grad_cam_img = generate_grad_cam(model, sample_img, true_label, preprocessors[name])
    plt.subplot(1, 4, i+1)
    plt.imshow(grad_cam_img)
    plt.title(f"{name} Grad-CAM")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Define preprocessors
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess,
    "Ensemble": efficientnet_preprocess  # Ensemble uses EfficientNetB0 preprocessing
}

# 🔥 Plot Grad-CAMs including Ensemble
plt.figure(figsize=(20, 8))  # Adjusted figure size for 5 subplots
for i, (name, model) in enumerate(models.items()):
    grad_cam_img = generate_grad_cam(model, sample_img, true_label, preprocessors[name])
    plt.subplot(1, 5, i+1)
    plt.imshow(grad_cam_img)
    plt.title(f"{name} Grad-CAM")
    plt.axis('off')

# Add Ensemble Grad-CAM by averaging individual model contributions
ensemble_grads = np.mean([generate_grad_cam(model, sample_img, true_label, preprocessors[name])[:, :, ::-1] 
                         for name, model in models.items()], axis=0)  # Convert BGR to RGB
plt.subplot(1, 5, 5)
plt.imshow(ensemble_grads)
plt.title("Ensemble Grad-CAM")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# ✅ Step 12: Accuracy and Loss Curves
for name, history in histories.items():
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train')
    plt.plot(history.history['val_accuracy'], label='Validation')
    plt.title(f'{name} Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.title(f'{name} Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# ✅ Step 13: Compare Validation Accuracies Across Models
plt.figure(figsize=(10, 6))
for name, history in histories.items():
    plt.plot(history.history['val_accuracy'], label=f'{name}')

plt.title('Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ✅ Step 12: Combined Accuracy and Loss Curves for All Models

# Plot all model accuracies on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label=f'{name} (val)')
plt.title('📈 Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot all model losses on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label=f'{name} (val)')
plt.title('📉 Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ Step 12: Combined Accuracy and Loss Curves for All Models

# Create a figure with two subplots side by side
plt.figure(figsize=(20, 5))

# Plot all model validation accuracies on the left subplot
plt.subplot(1, 2, 1)
for name, history in histories.items():
    if 'val_accuracy' in history.history:
        plt.plot(history.history['val_accuracy'], label=f'{name} (val)')
# Add ensemble validation accuracy (using Step 8 ensemble_acc as a proxy, adjust if ensemble history exists)
plt.axhline(y=ensemble_acc, color='r', linestyle='--', label='Ensemble (val)')
plt.title('Validation Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)

# Plot all model validation losses on the right subplot
plt.subplot(1, 2, 2)
for name, history in histories.items():
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label=f'{name} (val)')
# Add ensemble validation loss (placeholder; replace with ensemble history if available)
plt.axhline(y=np.mean([history.history['val_loss'][-1] for history in histories.values() if 'val_loss' in history.history]), 
            color='r', linestyle='--', label='Ensemble (val approx)')
plt.title('Validation Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ✅ Step 12: Combined Training Accuracy and Loss for All Models

# Plot all model training accuracies on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'accuracy' in history.history:
        plt.plot(history.history['accuracy'], label=f'{name} (train)')
plt.title('📈 Training Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot all model training losses on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'loss' in history.history:
        plt.plot(history.history['loss'], label=f'{name} (train)')
plt.title('📉 Training Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# ✅ Step 12: Combined Training Accuracy and Loss for All Models

# Plot all model training accuracies on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'accuracy' in history.history:
        plt.plot(history.history['accuracy'], label=f'{name} (train)')
# Add ensemble accuracy (using Step 8 ensemble_acc as a proxy, adjust if ensemble history exists)
plt.axhline(y=ensemble_acc, color='r', linestyle='--', label='Ensemble (train)')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Model Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plot all model training losses on one graph
plt.figure(figsize=(10, 5))
for name, history in histories.items():
    if 'loss' in history.history:
        plt.plot(history.history['loss'], label=f'{name} (train)')
# Add ensemble loss (placeholder; replace with ensemble history if available)
plt.axhline(y=np.mean([history.history['loss'][-1] for history in histories.values() if 'loss' in history.history]), 
            color='r', linestyle='--', label='Ensemble (train approx)')
plt.title('Model Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Model Loss')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ✅ Step 12: Combined Training Accuracy and Loss for All Models

# Create a figure with two subplots side by side
plt.figure(figsize=(20, 5))

# Plot all model training accuracies on the left subplot
plt.subplot(1, 2, 1)
for name, history in histories.items():
    if 'accuracy' in history.history:
        plt.plot(history.history['accuracy'], label=f'{name} (train)')
# Add ensemble accuracy (using Step 8 ensemble_acc as a proxy, adjust if ensemble history exists)
plt.axhline(y=ensemble_acc, color='r', linestyle='--', label='Ensemble (train)')
plt.title('Model Accuracy Comparison')
plt.xlabel('Epochs')
plt.ylabel('Model Accuracy')
plt.legend()
plt.grid(True)

# Plot all model training losses on the right subplot
plt.subplot(1, 2, 2)
for name, history in histories.items():
    if 'loss' in history.history:
        plt.plot(history.history['loss'], label=f'{name} (train)')
# Add ensemble loss (placeholder; replace with ensemble history if available)
plt.axhline(y=np.mean([history.history['loss'][-1] for history in histories.values() if 'loss' in history.history]), 
            color='r', linestyle='--', label='Ensemble (train approx)')
plt.title('Model Loss Comparison')
plt.xlabel('Epochs')
plt.ylabel('Model Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# ✅ Step 14: Accuracy/Loss Curves (Train vs Validation)
for name, history in histories.items():
    acc = history.history.get('accuracy', [])
    val_acc = history.history.get('val_accuracy', [])
    loss = history.history.get('loss', [])
    val_loss = history.history.get('val_loss', [])

    if acc and val_acc:
        plt.figure(figsize=(12, 4))

        # Accuracy plot
        plt.subplot(1, 2, 1)
        plt.plot(acc, label='Train Accuracy')
        plt.plot(val_acc, label='Val Accuracy')
        plt.title(f'{name} - Accuracy Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        # Loss plot
        plt.subplot(1, 2, 2)
        plt.plot(loss, label='Train Loss')
        plt.plot(val_loss, label='Val Loss')
        plt.title(f'{name} - Loss Curve')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# ✅ Step 15: Model Performance Comparison Table
metrics_table = []

for name, preds in model_preds.items():
    y_pred_labels = np.argmax(preds, axis=1)
    acc = accuracy_score(y_val, y_pred_labels)
    prec = precision_score(y_val, y_pred_labels, average='macro')
    rec = recall_score(y_val, y_pred_labels, average='macro')
    f1 = f1_score(y_val, y_pred_labels, average='macro')
    metrics_table.append([name, acc, prec, rec, f1])

# Add Ensemble
ensemble_preds = np.argmax(np.mean(list(model_preds.values()), axis=0), axis=1)
ensemble_acc = accuracy_score(y_val, ensemble_preds)
ensemble_prec = precision_score(y_val, ensemble_preds, average='macro')
ensemble_rec = recall_score(y_val, ensemble_preds, average='macro')
ensemble_f1 = f1_score(y_val, ensemble_preds, average='macro')
metrics_table.append(["Ensemble", ensemble_acc, ensemble_prec, ensemble_rec, ensemble_f1])

# Create DataFrame
df_metrics = pd.DataFrame(metrics_table, columns=["Model", "Accuracy", "Precision", "Recall", "F1-Score"])
df_metrics = df_metrics.sort_values("Accuracy", ascending=False)
print(df_metrics)


In [ ]:
# Optional: Visual Comparison
df_metrics.set_index("Model")[["Accuracy", "Precision", "Recall", "F1-Score"]].plot(kind="bar", figsize=(12, 6))
plt.title("📊 Model Performance Comparison")
plt.ylabel("Score")
plt.ylim(0, 1.05)
plt.grid(True)
plt.xticks(rotation=45)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()


In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Grad-CAM Function
def generate_grad_cam(model, image, label_index, preprocess=None):
    img_tensor = np.expand_dims(image, axis=0)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    # Detect last Conv layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break

    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + tf.keras.backend.epsilon())
    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    return heatmap  # Return raw heatmap for averaging

# Function to overlay heatmap on image
def overlay_heatmap(heatmap, image):
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image.astype(np.uint8), 0.6, heatmap_colored, 0.4, 0)
    return superimposed_img

# Define preprocessors
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess,
    "Ensemble": efficientnet_preprocess  # Using EfficientNetB0 preprocessing for consistency
}

# Sample image from validation set
sample_img = X_val_resized[0]
true_label = y_val[0]

# Generate Grad-CAM heatmaps for all models
heatmaps = {}
for name, model in models.items():
    print(f"🔍 Generating Grad-CAM heatmap for {name}...")
    heatmap = generate_grad_cam(model, sample_img, true_label, preprocess=preprocessors[name])
    heatmaps[name] = heatmap

# Compute ensemble heatmap by averaging individual heatmaps
ensemble_heatmap = np.mean(list(heatmaps.values()), axis=0)
ensemble_heatmap = np.maximum(ensemble_heatmap, 0) / np.max(ensemble_heatmap)  # Normalize

# Overlay heatmaps on the original image
plt.figure(figsize=(20, 8))
for i, (name, model) in enumerate(models.items()):
    grad_cam_img = overlay_heatmap(heatmaps[name], sample_img)
    plt.subplot(1, 5, i + 1)
    plt.imshow(grad_cam_img)
    plt.title(f"{name} Grad-CAM")
    plt.axis('off')

# Overlay ensemble heatmap
ensemble_grad_cam_img = overlay_heatmap(ensemble_heatmap, sample_img)
plt.subplot(1, 5, 5)
plt.imshow(ensemble_grad_cam_img)
plt.title("Ensemble Grad-CAM")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Grad-CAM Function
def generate_grad_cam(model, image, label_index, preprocess=None):
    img_tensor = np.expand_dims(image, axis=0)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    # Detect last Conv layer
    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break

    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + tf.keras.backend.epsilon())
    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    return heatmap

# Function to overlay heatmap on image
def overlay_heatmap(heatmap, image):
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(image.astype(np.uint8), 0.6, heatmap_colored, 0.4, 0)
    return superimposed_img

# Define preprocessors
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess,
    "Ensemble": efficientnet_preprocess  # Using EfficientNetB0 preprocessing for consistency
}

# Sample image from validation set
sample_img = X_val_resized[0]

# Get predictions for the sample image
heatmaps = {}
predictions = {}
for name, model in models.items():
    img_tensor = np.expand_dims(sample_img, axis=0)
    if preprocessors[name]:
        img_tensor = preprocessors[name](img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0
    pred = model.predict(img_tensor)
    predicted_label = np.argmax(pred)
    predictions[name] = predicted_label
    heatmap = generate_grad_cam(model, sample_img, predicted_label, preprocess=preprocessors[name])
    heatmaps[name] = heatmap

# Compute ensemble prediction and heatmap
ensemble_preds = np.mean([models[name].predict(np.expand_dims(sample_img, axis=0) if not preprocessors[name] else preprocessors[name](np.expand_dims(sample_img, axis=0))) for name in models], axis=0)
ensemble_label = np.argmax(ensemble_preds)
ensemble_heatmap = np.mean(list(heatmaps.values()), axis=0)
ensemble_heatmap = np.maximum(ensemble_heatmap, 0) / np.max(ensemble_heatmap)

# Overlay heatmaps on the original image
plt.figure(figsize=(20, 8))
for i, (name, model) in enumerate(models.items()):
    grad_cam_img = overlay_heatmap(heatmaps[name], sample_img)
    plt.subplot(1, 5, i + 1)
    plt.imshow(grad_cam_img)
    plt.title(f"{name} Grad-CAM ")
    plt.axis('off')

# Overlay ensemble heatmap
ensemble_grad_cam_img = overlay_heatmap(ensemble_heatmap, sample_img)
plt.subplot(1, 5, 5)
plt.imshow(ensemble_grad_cam_img)
plt.title(f"Ensemble Grad-CAM")
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Guided Backpropagation Function
def guided_backprop(model, image, label_index, preprocess=None):
    img_tensor = np.expand_dims(image, axis=0).astype(np.float32)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    img_tensor = tf.convert_to_tensor(img_tensor, dtype=tf.float32)

    @tf.custom_gradient
    def guided_relu(x):
        def grad(dy):
            return tf.cast(dy > 0, "float32") * tf.cast(x > 0, "float32") * dy
        return tf.nn.relu(x), grad

    layer_dict = [layer for layer in model.layers if hasattr(layer, 'activation')]
    for layer in layer_dict:
        if layer.activation == tf.keras.activations.relu:
            layer.activation = guided_relu

    with tf.GradientTape() as tape:
        tape.watch(img_tensor)
        preds = model(img_tensor, training=False)
        loss = preds[:, label_index]

    grads = tape.gradient(loss, img_tensor)[0]
    guided_grads = tf.cast(grads > 0, "float32") * tf.cast(img_tensor > 0, "float32") * grads
    guided_grads = tf.reduce_mean(guided_grads, axis=-1)  # Ensure single channel
    guided_grads = tf.squeeze(guided_grads)  # Remove any extra dimensions
    guided_grads = cv2.resize(guided_grads.numpy(), (224, 224))
    return guided_grads

# Grad-CAM Function
def generate_grad_cam(model, image, label_index, preprocess=None):
    img_tensor = np.expand_dims(image, axis=0).astype(np.float32)
    if preprocess:
        img_tensor = preprocess(img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0

    last_conv_layer = None
    for layer in reversed(model.layers):
        if isinstance(layer, tf.keras.layers.Conv2D):
            last_conv_layer = layer.name
            break

    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_tensor)
        loss = predictions[:, label_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + tf.keras.backend.epsilon())
    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    return heatmap

# Function to overlay heatmap on image
def overlay_heatmap(heatmap, image, guided_grads=None):
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    if guided_grads is not None:
        # Ensure guided_grads is 2D and of type uint8
        if len(guided_grads.shape) > 2:
            guided_grads = guided_grads[..., 0]  # Take the first channel if multi-channel
        guided_grads_normalized = cv2.normalize(guided_grads, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
        guided_grads_colored = cv2.applyColorMap(guided_grads_normalized, cv2.COLORMAP_JET)
        heatmap_colored = cv2.addWeighted(heatmap_colored, 0.7, guided_grads_colored, 0.3, 0)
    superimposed_img = cv2.addWeighted(image.astype(np.uint8), 0.6, heatmap_colored, 0.4, 0)
    return superimposed_img

# Define preprocessors
preprocessors = {
    "CustomCNN": None,
    "VGG16": vgg16_preprocess,
    "MobileNetV2": mobilenet_preprocess,
    "EfficientNetB0": efficientnet_preprocess,
    "Ensemble": efficientnet_preprocess
}

# Sample image from validation set
sample_img = X_val_resized[0]

# Get predictions and heatmaps
heatmaps = {}
guided_grads = {}
predictions = {}
for name, model in models.items():
    img_tensor = np.expand_dims(sample_img, axis=0).astype(np.float32)
    if preprocessors[name]:
        img_tensor = preprocessors[name](img_tensor.copy())
    else:
        img_tensor = img_tensor / 255.0
    pred = model.predict(img_tensor)
    predicted_label = np.argmax(pred)
    predictions[name] = predicted_label
    heatmap = generate_grad_cam(model, sample_img, predicted_label, preprocess=preprocessors[name])
    guided_grad = guided_backprop(model, sample_img, predicted_label, preprocess=preprocessors[name])
    heatmaps[name] = heatmap
    guided_grads[name] = guided_grad

# Compute ensemble prediction and weighted heatmap
ensemble_preds = np.mean([models[name].predict(np.expand_dims(sample_img, axis=0).astype(np.float32) if not preprocessors[name] else preprocessors[name](np.expand_dims(sample_img, axis=0).astype(np.float32))) for name in models], axis=0)
ensemble_label = np.argmax(ensemble_preds)
confidences = [pred.max() for pred in [models[name].predict(np.expand_dims(sample_img, axis=0).astype(np.float32) if not preprocessors[name] else preprocessors[name](np.expand_dims(sample_img, axis=0).astype(np.float32))) for name in models]]
ensemble_heatmap = np.average(list(heatmaps.values()), weights=confidences, axis=0)
ensemble_heatmap = np.maximum(ensemble_heatmap, 0) / np.max(ensemble_heatmap)
ensemble_guided_grad = np.average(list(guided_grads.values()), weights=confidences, axis=0)

# Overlay heatmaps
plt.figure(figsize=(20, 8))
for i, (name, model) in enumerate(models.items()):
    grad_cam_img = overlay_heatmap(heatmaps[name], sample_img, guided_grads[name])
    plt.subplot(1, 5, i + 1)
    plt.imshow(grad_cam_img)
    plt.title(f"{name} Grad-CAM (Pred: {predictions[name]})")
    plt.axis('off')

# Overlay ensemble heatmap
ensemble_grad_cam_img = overlay_heatmap(ensemble_heatmap, sample_img, ensemble_guided_grad)
plt.subplot(1, 5, 5)
plt.imshow(ensemble_grad_cam_img)
plt.title(f"Ensemble Grad-CAM (Pred: {ensemble_label})")
plt.axis('off')

plt.tight_layout()
plt.show()